Grab fablib manager

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress
import json
import shlex
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

Find a site

In [ ]:
# find a site with appropriate cores
cores_per_node = 16
ram_per_node = 32
disk_per_node = 20
site1 = fablib.get_random_site(
    filter_function=lambda x: x["cores_available"] > cores_per_node * 3 and x["ram_available"] > ram_per_node * 3 and x["disk_available"] > disk_per_node * 3 and x["nic_connectx_6_available"] > 2
)
hosts = [f"{site1.lower()}-c1", f"{site1.lower()}-r1", f"{site1.lower()}-p1"]
fablib.show_site(site1, 'json')

In [ ]:
slice_name = 'ndn-poc-sjzhu2'

node1_name='Node1'
node2_name='Node2'

network_name='net1'
nic_name = 'nic1'
model_name = 'NIC_Basic'

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

for h in hosts:

    node = slice.add_node(name=f"{h}-a", cores=cores_per_node, ram=ram_per_node, disk=disk_per_node, site=site1, image='default_ubuntu_22')
    node.add_fabnet(nic_type='NIC_ConnectX_6')
    node.add_post_boot_upload_directory("scripts")
    node.add_post_boot_execute("./scripts/install_dependencies.sh")
    node.add_post_boot_execute("./scripts/install-ndn-dpdk.sh")

#Submit Slice Request
slice.submit();

Here are all of the settings for the various NDN services that need to be booted up.

In [ ]:
FW_ARGS = {
  'mempool': {
    'DIRECT': { 'capacity': 524287, 'dataroom': 9146 },
    'INDIRECT': { 'capacity': 524287 },
  },
  'fib': {
    'capacity': 4095,
    'startDepth': 8,
  },
  'pcct': {
    'pcctCapacity': 65535,
    'csMemoryCapacity': 20000,
    'csIndirectCapacity': 20000,
  }
}
MEMIF_ARGS = {
    'scheme': "memif",
    'socketName': "/run/ndn/fileserver.sock",
    'id': 0,
    'role': "server",
    'dataroom': 9000
}
FS_ACTIVATE = {
    'eal': {
        'memPerNuma': {'0': 4*1024},
        'filePrefix': 'producer',
    },
    'mempool': {
        'DIRECT': {'capacity': 2**16-1, 'dataroom': 9200},
        'INDIRECT': {'capacity': 2**16-1},
        'PAYLOAD': {'capacity': 2**16-1, 'dataroom': 9200},
    },
    'face': {
        'scheme': 'memif',
        'socketName': MEMIF_ARGS['socketName'],
        'id': MEMIF_ARGS['id'],
        'dataroom': 9000,
        'role': 'client',
    },
    'fileServer': {
        'mounts': [
            {'prefix': "/producer/usr-local-share", 'path': "/usr/local/share"}
        ],
        'segmentLen': 6 * 1024,
    },
}

adj_list = {
    hosts[0]: [hosts[1]],
    hosts[1]: [hosts[0], hosts[2]],
    hosts[2]: [hosts[1]],
}

forwarder_hashes = {}
face_hashes = {}

In [ ]:
# add eth ports to each node
for h in hosts:
    node = slice.get_node(f"{h}-a")
    output = node.execute(f'''
        echo {shlex.quote(json.dumps(FW_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ activate-forwarder
        ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-eth-port --netif enp7s0np0 --xdp --mtu 1500
        ''')
    forwarder_hashes[h] = json.loads(output[0].split("\n")[-2])

# add interface between nodes in adj_list
for h in adj_list:
    local_mac = forwarder_hashes[h]['macAddr']
    node = slice.get_node(f"{h}-a")
    for remote in adj_list[h]:
        remote_mac = forwarder_hashes[remote]['macAddr']
        output = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-ether-face --local {local_mac} --remote {remote_mac}")
        face_hashes[f"{h}:{remote}"] = json.loads(output[0])

In [ ]:
print(forwarder_hashes)
print(face_hashes)
c_to_r = face_hashes[f"{hosts[0]}:{hosts[1]}"]
print(f"{hosts[0]}:{hosts[1]}")

In [ ]:
# add fileserver configuration to producer
producer_name = hosts[2] # hardcode this for now
producer_node = slice.get_node(f"{producer_name}-a")
output = producer_node.execute(f'''
    MEMIF_FACE=$(echo {shlex.quote(json.dumps(MEMIF_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-face)
    ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh $(echo $MEMIF_FACE | jq -r .id)
    ''')
# start fileserver instance
output = producer_node.execute(f'''
    sudo ndndpdk-ctrl --gqlserver http://127.0.0.1:3031/ systemd start
    echo {shlex.quote(json.dumps(FS_ACTIVATE))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3031 activate-fileserver
    ''')

In [ ]:
# add FIB entries to point to producer
c_node = slice.get_node(f"{hosts[0]}-a")
c_to_r = face_hashes[f"{hosts[0]}:{hosts[1]}"]
output = c_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh {c_to_r['id']}")

r_node = slice.get_node(f"{hosts[1]}-a")
r_to_p = face_hashes[f"{hosts[1]}:{hosts[2]}"]
r_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh {r_to_p['id']}")

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

end_date = (datetime.now(timezone.utc) + timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S %z")
try:
    slice = fablib.get_slice(slice_name)
    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
slice.update()

In [ ]:
slice = fablib.get_slice(slice_name)
slice.show()